In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

In [75]:
mnist_data = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [76]:
mnist_data.count
mnist_data.index

<function Datasets.index>

In [239]:
input_size = 784
output_size = 10
hidden_layer_size = 200

tf.reset_default_graph() # This is need to reset to any memorized variables

In [240]:
inputs = tf.placeholder(tf.float32, [None, input_size], name='inputs')
targets = tf.placeholder(tf.float32, [None, output_size], name='targets')

In [241]:
print(inputs.name)
print(targets.name)

inputs:0
targets:0


In [242]:
weights_1 = tf.get_variable('weights_1', [input_size, hidden_layer_size])
biases_1 = tf.get_variable('biases_1', [hidden_layer_size])

outputs_1 = tf.nn.sigmoid(tf.matmul(inputs, weights_1) + biases_1)

In [243]:
weights_2 = tf.get_variable('weights_2', [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable('biases_2', [hidden_layer_size])

outputs_2 = tf.nn.sigmoid(tf.matmul(outputs_1, weights_2) + biases_2)

In [244]:
weights_3 = tf.get_variable('weights_3', [hidden_layer_size, hidden_layer_size])
biases_3 = tf.get_variable('biases_3', [hidden_layer_size])

outputs_3 = tf.nn.sigmoid(tf.matmul(outputs_2, weights_3) + biases_3)

In [245]:
weights_4 = tf.get_variable('weights_4', [hidden_layer_size, hidden_layer_size])
biases_4 = tf.get_variable('biases_4', [hidden_layer_size])

outputs_4 = tf.nn.sigmoid(tf.matmul(outputs_3, weights_4) + biases_4)

In [246]:
weights_5 = tf.get_variable('weights_5', [hidden_layer_size, output_size])
biases_5 = tf.get_variable('biases_5', [output_size])

outputs = tf.matmul(outputs_4, weights_5) + biases_5

In [247]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)

In [248]:
optimize = tf.train.AdamOptimizer(learning_rate=0.02).minimize(mean_loss)

In [249]:
out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))

In [250]:
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

In [251]:
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

In [252]:
batch_size = 1
batch_numbers = mnist_data.train._num_examples // batch_size
max_epochs = 15
prev_validation_loss = 9999999.

In [ ]:
for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.0
    for batch_counter in range(batch_numbers):
        input_batch, target_batch = mnist_data.train.next_batch(batch_size) # Here we get in dataset the batch of size X with inputs and targets
        
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict={inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss # Sum the errors per batch
    
    curr_epoch_loss /= batch_numbers # Mean loss for the batchs
    
    #Validation part
    input_batch, target_batch = mnist_data.validation.next_batch(mnist_data.validation._num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                   feed_dict={inputs: input_batch, targets: target_batch})
    
    print("Epoch " + str(epoch_counter + 1))
    print("Training Loss %.3f" % curr_epoch_loss)
    print("Validation Loss %.3f" % validation_loss)
    print("Validation accuracy %.3f percent" % (validation_accuracy * 100))
    print("\n\n")
    
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss = validation_loss
    
print("End of Training")

Epoch 1
Training Loss 2.332
Validation Loss 2.345
Validation accuracy 10.020 percent



Epoch 2
Training Loss 2.330
Validation Loss 2.327
Validation accuracy 9.900 percent





In [ ]:
#Test Model
input_batch, target_batch = mnist_data.test.next_batch(mnist_data.test._num_examples)
test_accuracy = sess.run([accuracy], feed_dict={inputs: input_batch, targets: target_batch})

print(test_accuracy)

print("The accuracy from your test model is %.2f" % (test_accuracy[0] * 100))

In [ ]:
print(tf.get_default_graph().as_graph_def())